In [6]:
#include <uWS/uWS.h>
#include <iostream>
#include "json.hpp"
#include <math.h>
#include "FusionEKF.h"
#include "tools.h"

using namespace std;
#include "Eigen/Dense"
using Eigen::MatrixXd;
using Eigen::VectorXd;
using std::vector;

In [12]:
const double PI = 3.141592653589793238463;


/*
 * Constructor.
 */
FusionEKF::FusionEKF() {
    is_initialized_ = false;

    previous_timestamp_ = 0;

    // initializing matrices
    R_laser_ = MatrixXd(2, 2);
    R_radar_ = MatrixXd(3, 3);
    H_laser_ = MatrixXd(2, 4);
    Hj_ = MatrixXd(3, 4);


    //measurement covariance matrix - laser
    R_laser_ << 0.0225, 0,
                0, 0.0225;

    //measurement covariance matrix - radar
    R_radar_ << 0.09, 0, 0,
            0, 0.0009, 0,
            0, 0, 0.09;
    H_laser_ << 1, 0, 0, 0,
                0, 1, 0, 0;

    Hj_ <<  0, 0, 0, 0,
            0, 0, 0, 0,
            0, 0, 0, 0;

    /**
    TODO:
      * Finish initializing the FusionEKF.
      * Set the process and measurement noises
    */
    ekf_.P_ = MatrixXd(4, 4);
    ekf_.P_ << 1, 0, 0, 0,
            0, 1, 0, 0,
            0, 0, 1, 0,
            0, 0, 0, 1;

    ekf_.F_ = MatrixXd(4, 4);
    ekf_.F_ << 1, 0, 1, 0,
            0, 1, 0, 1,
            0, 0, 1, 0,
            0, 0, 0, 1;

    ekf_.Q_ = MatrixXd(4, 4);
    ekf_.Q_ << 1, 0, 1, 0,
               0, 1, 0, 1,
               0, 0, 1, 0,
               0, 0, 0, 1;

    ekf_.x_ = VectorXd(4);
    ekf_.x_ << 0, 0, 0, 0;


    float noise_ax = 9;
    float noise_ay = 9;

}

input_line_14:8:12: error: qualified reference to 'FusionEKF' is a constructor name rather than a type in
      this context
FusionEKF::FusionEKF() {
           ^
input_line_14:8:23: error: expected ';' after expression
FusionEKF::FusionEKF() {
                      ^
                      ;
input_line_14:9:5: error: use of undeclared identifier 'is_initialized_'
    is_initialized_ = false;
    ^
input_line_14:11:5: error: use of undeclared identifier 'previous_timestamp_'
    previous_timestamp_ = 0;
    ^
input_line_14:14:5: error: use of undeclared identifier 'R_laser_'
    R_laser_ = MatrixXd(2, 2);
    ^
input_line_14:15:5: error: use of undeclared identifier 'R_radar_'
    R_radar_ = MatrixXd(3, 3);
    ^
input_line_14:16:5: error: use of undeclared identifier 'H_laser_'
    H_laser_ = MatrixXd(2, 4);
    ^
input_line_14:17:5: error: use of undeclared identifier 'Hj_'
    Hj_ = MatrixXd(3, 4);
    ^
input_line_14:21:5: error: use of undeclared identifier 'R_laser_'
    R_laser_ 

ename: evalue

In [ ]:
/**
* Destructor.
*/
FusionEKF::~FusionEKF() {}

void FusionEKF::ProcessMeasurement(const MeasurementPackage &measurement_pack) {


    /*****************************************************************************
     *  Initialization
     ****************************************************************************/
    if (!is_initialized_) {
        /**
        TODO:
          * Initialize the state ekf_.x_ with the first measurement.
          * Create the covariance matrix.
          * Remember: you'll need to convert radar from polar to cartesian coordinates.
        */
        // first measurement
        cout << "EKF: " << endl;
//    ekf_.x_ = VectorXd(4);
        ekf_.x_ << 1, 1, 1, 1;

        if (measurement_pack.sensor_type_ == MeasurementPackage::RADAR) {
            /**
            Convert radar from polar to cartesian coordinates and initialize state.
            */
            float rho = measurement_pack.raw_measurements_[0];
            float theta = measurement_pack.raw_measurements_[1];

            if (theta > PI)
                theta = fmod((theta - PI), 2 * PI) - PI;

            if (theta < -PI)
                theta = fmod((theta + PI), 2 * PI) + PI;

            float x = rho * cos(theta);
            float y = rho * sin(theta);

            ekf_.x_ << x, y, 0, 0;

        } else if (measurement_pack.sensor_type_ == MeasurementPackage::LASER) {
            /**
            Initialize state.
            */
            ekf_.x_ <<
                    measurement_pack.raw_measurements_[0],
                    measurement_pack.raw_measurements_[1],
                    0,
                    0;
        }
        previous_timestamp_ = measurement_pack.timestamp_;

        // done initializing, no need to predict or update
        is_initialized_ = true;
        return;
    }

    /*****************************************************************************
     *  Prediction
     ****************************************************************************/

    /**
     TODO:
       * Update the state transition matrix F according to the new elapsed time.
        - Time is measured in seconds.
       * Update the process noise covariance matrix.
       * Use noise_ax = 9 and noise_ay = 9 for your Q matrix.
     */
    float delta_t = measurement_pack.timestamp_ - previous_timestamp_ / 1000000.0;

    ekf_.F_(0, 2) = delta_t;
    ekf_.F_(1, 3) = delta_t;

    float dt_2 = pow(delta_t, 2);
    float dt_3 = pow(delta_t, 3);
    float dt_4 = pow(delta_t, 4);


    previous_timestamp_ = measurement_pack.timestamp_;
    float noise_ax = 9.0;
    float noise_ay = 9.0;
    //set the process covariance matrix Q
    ekf_.Q_ = MatrixXd(4, 4);
    ekf_.Q_ << dt_4 / 4 * noise_ax, 0, dt_3 / 2 * noise_ax, 0,
            0, dt_4 / 4 * noise_ay, 0, dt_3 / 2 * noise_ay,
            dt_3 / 2 * noise_ax, 0, dt_2 * noise_ax, 0,
            0, dt_3 / 2 * noise_ay, 0, dt_2 * noise_ay;
    ekf_.Predict();
    /*****************************************************************************
     *  Update
     ****************************************************************************/

    /**
     TODO:
       * Use the sensor type to perform the update step.
       * Update the state and covariance matrices.
     */


    if (measurement_pack.sensor_type_ == MeasurementPackage::RADAR) {
        // Radar updates
        Hj_ << tools.CalculateJacobian(ekf_.x_);

        ekf_.H_ = Hj_;

        ekf_.R_ = R_radar_;
        ekf_.UpdateEKF(measurement_pack.raw_measurements_);
    } else {
        // Laser updates
        ekf_.H_ = H_laser_;
        ekf_.R_ = R_laser_;
        ekf_.Update(measurement_pack.raw_measurements_);
    }

    // print the output
    cout << "x_ = " << ekf_.x_ << endl;
    cout << "P_ = " << ekf_.P_ << endl;
}